In [1]:
import numpy as np

# American Option Pricing
Binomial Options Pricing Model to price american options (the ability to exercise the option early)

In [2]:
def american_option(dp, S0, K, T, r, q, sigma, n_steps, dt, opttype):
    
    disc = np.exp(-r*dt)
    
    if (S0, n_steps) in dp:
        return dp[(S0, n_steps)]

    if n_steps == 0:
        
        if opttype == 'C':
            dp[(S0, n_steps)] = max(S0 - K, 0)
        elif  opttype == 'P':
            dp[(S0, n_steps)] = max(K - S0, 0)
            
        return dp[(S0, n_steps)]
    else:
        u = np.exp(sigma * np.sqrt(dt))
        d = np.exp(-sigma * np.sqrt(dt))
        p = (np.exp((r-q)*dt) - d)/(u-d)
        fu = american_option(dp, S0 * u, K, T - dt, r, q, sigma, n_steps - 1, dt, opttype)
        fd = american_option(dp, S0 * d, K, T - dt, r, q, sigma, n_steps - 1, dt, opttype)
        f = p * fu + (1 - p) * fd
        
        if opttype == 'C':
            dp[(S0, n_steps)] = max(disc*f, S0-K)
        elif  opttype == 'P':
            dp[(S0, n_steps)] = max(disc*f, K-S0)

        
        return dp[(S0, n_steps)]

### Run BOTH if calculating for PUT Option

In [3]:
# Parameters
S0 = 100.0
K = 100.0
T = 1.0
r = 0.05
q = 0.08
sigma = 0.35
opttype = 'P'

n_steps = 100 # steps
dt = T / n_steps

In [4]:
# Pricing American Put Option Using a 100 Step Binomial Tree
dp = {}
option_price = american_option(dp, S0, K, T, r, q, sigma, n_steps, dt, opttype)
print(f"Price of American Put Option = ${option_price}")

Price of American Put Option = $14.4635131795728


### Run BOTH if calculating for CALL Option

In [5]:
# Parameters
S0 = 100.0
K = 100.0
T = 1.0
r = 0.05
q = 0.08
sigma = 0.35
opttype = 'C'

n_steps = 100 # steps
dt = T / n_steps

In [6]:
# Pricing American Call Option Using a 100 Step Binomial Tree
dp = {}
option_price = american_option(dp, S0, K, T, r, q, sigma, n_steps, dt, opttype)
print(f"Price of American Call Option = ${option_price}")

Price of American Call Option = $12.1188359407854


In [ ]:
dp # for n_steps = 100

# Delta


In [7]:
def find_delta(S0, K, T, r, q, sigma, n_steps, dt, dp, opttype):
    u = np.exp(sigma * np.sqrt(dt))
    d = np.exp(-sigma * np.sqrt(dt))
    f1 = dp[(S0 * u, n_steps-1)]
    f0 = dp[(S0 * d, n_steps-1)]
    return (f1 - f0) / (S0 * u - S0 * d)

In [8]:
delta = find_delta(S0, K, T, r, q, sigma, n_steps, dt, dp, opttype)
print(f"Delta = {delta}")

Delta = 0.5235945884300923


# Gamma

In [9]:
def find_gamma(S0, K, T, r, q, sigma, n_steps, dt, dp, opttype):
    u = np.exp(sigma * np.sqrt(dt))
    d = np.exp(-sigma * np.sqrt(dt))

    S1t1 = S0 * u
    S0t1 = S0 * d

    delta2 = find_delta(S1t1, K, T - dt, r, q, sigma, n_steps - 1, dt, dp, opttype)
    delta1 = find_delta(S0t1, K, T - dt, r, q, sigma, n_steps - 1, dt, dp, opttype)

    return (delta2 - delta1) / (S1t1 - S0t1)

In [10]:
gamma = find_gamma(S0, K, T, r, q, sigma, n_steps, dt, dp, opttype)
print(f"Gamma = {gamma}")

Gamma = 0.011803751155039987


# Theta

In [11]:
def find_theta(S0, K, T, r, q, sigma, n_steps, dt, dp, opttype):
    u = np.exp(sigma * np.sqrt(dt))
    d = np.exp(-sigma * np.sqrt(dt))
    
    f50t100 = dp[(S0 * u**50 * d**50, 0)]
    f0t0 = dp[(S0 , n_steps)]

    return (f50t100 - f0t0) / 100 * dt

In [12]:
theta = find_theta(S0, K, T, r, q, sigma, n_steps, dt, dp, opttype)
print(f"Theta = {theta}")

Theta = -0.0012118835940785372


# Vega

In [13]:
def find_vega(S0, K, T, r, q, sigma, n_steps, dt, dp, opttype):
    
    # Run binomial tree on a different sigma (sigmaPrime)
    deltaSigma = 0.2
    sigmaPrime = sigma + deltaSigma
    u = np.exp(sigma * np.sqrt(dt))
    d = np.exp(-sigma * np.sqrt(dt))
    
    dtPrime = {}
    
    fprime = american_option(dtPrime, S0, K, T, r, q, sigmaPrime, n_steps, dt, opttype)
#     print(fprime)
    f = dp[(S0, n_steps)]
#     print(f)


    return (fprime - f) / (deltaSigma*100)

In [14]:
vega = find_vega(S0, K, T, r, q, sigma, n_steps, dt, dp, opttype)
print(f"Vega = {vega}")

Vega = 0.3685443678166533


# Rho

In [15]:
def find_rho(S0, K, T, r, q, sigma, n_steps, dt, dp, opttype):
    
    # Run binomial tree on a different sigma (sigmaPrime)
    deltaR = 0.1
    rPrime = r + deltaR
    u = np.exp(sigma * np.sqrt(dt))
    d = np.exp(-sigma * np.sqrt(dt))
    
    dtPrime = {}
    
    fprime = american_option(dtPrime, S0, K, T, rPrime, q, sigma, n_steps, dt, opttype)
#     print(fprime)
    f = dp[(S0, n_steps)]
#     print(f)


    return (fprime - f) / (deltaR*100)

In [16]:
rho = find_rho(S0, K, T, r, q, sigma, n_steps, dt, dp, opttype)
print(f"Rho = {rho}")

Rho = 0.3619865542935468
